# OpenAI 외 도구 호출 에이전트(Tool Calling Agent)

OpenAI 외에도 `Anthropic`, `Google Gemini`, `Together.ai`, `Ollama`, `Mistral`과 같은 더 광범위한 공급자 구현을 지원합니다.

이번 챕터에서는 다양한 LLM 을 사용하여 도구 호출 에이전트를 생성하고 실행하는 방법을 살펴보겠습니다.

**참고 링크**

- [LangChain 공식 도큐먼트](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/tool_calling/)

In [2]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [3]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH15-Agents")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH15-Agents


In [4]:
from langchain.tools import tool
from typing import List, Dict
from langchain_teddynote.tools import GoogleNews


# 도구 정의
@tool
def search_news(query: str) -> List[Dict[str, str]]:
    """Search Google News by input keyword"""
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)


print(f"도구 이름: {search_news.name}")
print(f"도구 설명: {search_news.description}")

도구 이름: search_news
도구 설명: Search Google News by input keyword


In [5]:
# tools 정의
tools = [search_news]

## Agent 용 프롬프트 생성

- `chat_history` : 이전 대화 내용을 저장하는 변수 (멀티턴을 지원하지 않는다면, 생략 가능합니다.)
- `agent_scratchpad` : 에이전트가 임시로 저장하는 변수
- `input` : 사용자의 입력

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent

# 프롬프트 생성
# 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공합니다. (도구의 이름과 역할을 입력)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `search_news` tool for searching keyword related news.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

RuntimeError: no validator found for <class 'langchain_core.callbacks.base.BaseCallbackHandler'>, see `arbitrary_types_allowed` in Config

## Tool Calling 을 지원하는 다양한 LLM 목록

실습 진행을 위해서는 아래 내용을 설정해야 합니다.

**Anthropic**

- [Anthropic API 키 발급 관련](https://console.anthropic.com/settings/keys)
- `.env` 파일 내 `ANTHROPIC_API_KEY` 에 발급받은 키를 설정하세요

**Gemini**

- [Gemini API 키 발급 관련](https://aistudio.google.com/app/apikey?hl=ko)
- `.env` 파일 내 `GOOGLE_API_KEY` 에 발급받은 키를 설정하세요

**Together AI**

- [Together AI API 키 발급 관련](https://api.together.ai/)
- `.env` 파일 내 `TOGETHER_API_KEY` 에 발급받은 키를 설정하세요

**Ollama**

- [Ollama Tool Calling 지원 모델 리스트](https://ollama.com/search?c=tools)
- [이번 실습에 사용할 llama3.1 모델](https://ollama.com/library/llama3.1)
- 터미널 창에 `ollama pull llama3.1` 명령어를 입력하여 모델을 다운로드 받습니다.
- 이전에 Ollama 를 사용하지 않았다면, [Ollama](https://wikidocs.net/233805) 를 참고해 주세요.

langchain-ollama 설치를 한 뒤 진행해 주세요.

In [1]:
!pip install -qU langchain-ollama==0.1.3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.19 requires langchain-core<1.0.0,>=0.3.35, but you have langchain-core 0.2.43 which is incompatible.
langchain-anthropic 0.2.4 requires langchain-core<0.4.0,>=0.3.15, but you have langchain-core 0.2.43 which is incompatible.
langchain-cohere 0.3.4 requires langchain-core<0.4.0,>=0.3.27, but you have langchain-core 0.2.43 which is incompatible.
langchain-community 0.3.14 requires langchain-core<0.4.0,>=0.3.29, but you have langchain-core 0.2.43 which is incompatible.
langchain-elasticsearch 0.3.0 requires langchain-core<0.4.0,>=0.3.0, but you have langchain-core 0.2.43 which is incompatible.
langchain-experimental 0.3.4 requires langchain-core<0.4.0,>=0.3.28, but you have langchain-core 0.2.43 which is incompatible.
langchain-google-genai 2.0.8 requires langchain-core<0.4.0,>=0.3.27, but you have lang

In [ ]:
from langchain_anthropic import ChatAnthropic
from langchain_core.pydantic_v1 import BaseModel
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from langchain_ollama.chat_models import ChatOllama
import os

# GPT-4o-mini
gpt = ChatOpenAI(model="gpt-4o-mini")

# Claude-3-5-sonnet
claude = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0)

# Gemini-1.5-pro-latest
gemini = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)

# Llama-3.1-70B-Instruct-Turbo
llama = ChatOpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=os.environ["TOGETHER_API_KEY"],
    model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
)

# Llama-3.1
ollama = ChatOllama(model="llama3.1", temperature=0)

# Qwen2.5 7B (한글 성능 괜찮은 편)
qwen = ChatOllama(
    model="qwen2.5:latest",
)

LLM 기반으로 Agent 를 생성합니다.

In [ ]:
from langchain.agents import create_tool_calling_agent

# Agent 생성
gpt_agent = create_tool_calling_agent(gpt, tools, prompt)
claude_agent = create_tool_calling_agent(claude, tools, prompt)
gemini_agent = create_tool_calling_agent(gemini, tools, prompt)
llama_agent = create_tool_calling_agent(llama, tools, prompt)
ollama_agent = create_tool_calling_agent(ollama, tools, prompt)
qwen_agent = create_tool_calling_agent(qwen, tools, prompt)

## AgentExecutor 생성 후 실행 및 결과 확인



In [ ]:
from langchain.agents import AgentExecutor

# gpt_agent 실행
agent_executor = AgentExecutor(
    agent=gpt_agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)

result = agent_executor.invoke({"input": "AI 투자와 관련된 뉴스를 검색해 주세요."})

print("Agent 실행 결과:")
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `search_news` with `{'query': 'AI 투자'}`


[{'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTE53R2x6ek5lSXFBZl9WVnVmZ0VVUXZ4MWlJRkF6cE5WbENra0g4V0ZJUlVtMDhjUXRET3QzbGFQZ2xESEFOSHNNcFlNc3dDc0FvaFFoa0UtRFBwZ9IBVEFVX3lxTE5DQlR5VTh4R2xWM0hXQ2NfbHlnUEpDWUhXbW1aRG1VNUJQakJNVFk2MC1sNUVKWkxibEF0WjU4aS10V0tlczlRX2ROUFJFcTNnSnZJTQ?oc=5', 'content': "'마윈 매직'인가…中 빅테크들, AI 투자 러시 - 한국경제"}, {'url': 'https://news.google.com/rss/articles/CBMiT0FVX3lxTE5SOFY4M09KOG40VmxLX2xndTNoeFo0b3M0eENwd2JEbk1pSkpYdU1lQnQ3OUl0ZThTckdqNDRfblkzTklxTllDdjJzWE5IX1k?oc=5', 'content': '"중국, 6년간 AI에 2천조원 투자"…미중 간 \'쩐의 전쟁\' 가열 - ZUM 뉴스'}, {'url': 'https://news.google.com/rss/articles/CBMigAFBVV95cUxPdFktakY3cEZNVUUyYVY4bEY3QnZxdnpVeGxjLWYwYmhLNm9fYV90N2ZLVTBNODdHU2JQeUJFMzVWNFdsc1VfLXlqbWdjOC1PTG1nTHZYeHRoYmFrUFBPYWRGLWgxcWkxMGNRN1lnT0dkS1RsR2VNdVNTNS1KWExIQw?oc=5', 'content': '[VC’s Pick]투자 유치 성공한 ‘AI 의료 솔루션’들…글로벌 시장 정조준 - 이데일리'}, {'url': 'https://news.google.com/

다양한 llm을 사용하여 에이전트를 실행합니다.

다음은 입력받은 llm을 사용하여 Agent 를 생성하고 실행하여 결과를 출력하는 함수입니다.

In [ ]:
def execute_agent(llm, tools, input_text, label):
    agent = create_tool_calling_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools, verbose=False)
    result = executor.invoke({"input": input_text})
    print(f"[{label}] 결과입니다.")
    if isinstance(result["output"], list) and len(result["output"]) > 0:
        for item in result["output"]:
            if "text" in item:
                print(item["text"])
    elif isinstance(result["output"], str):
        print(result["output"])
    else:
        print(result["output"])

각 llm 별로 에이전트를 생성하고 실행하여 결과를 출력합니다.

In [ ]:
query = (
    "AI 투자와 관련된 뉴스를 검색하고, 결과를 Instagram 게시글 형식으로 작성해 주세요."
)

In [ ]:
# gpt
execute_agent(gpt, tools, query, "gpt")

[gpt] 결과입니다.
📈✨ **AI 투자, 미래를 선도하다!** ✨📈

최근 AI 분야에 대한 투자 열풍이 불고 있습니다! 🚀

1️⃣ **'마윈 매직'인가…中 빅테크들, AI 투자 러시**  
중국의 주요 기업들이 AI 기술에 대한 대규모 투자를 진행하고 있습니다. [자세히 보기](https://news.google.com/rss/articles/CBMiWkFVX3lxTE53R2x6ek5lSXFBZl9WVnVmZ0VVUXZ4MWlJRkF6cE5WbENra0g4V0ZJUlVtMDhjUXRET3QzbGFQZ2xESEFOSHNNcFlNc3dDc0FvaFFoa0UtRFBwZ9IBVEFVX3lxTE5DQlR5VTh4R2xWM0hXQ2NfbHlnUEpDWUhXbW1aRG1VNUJQakJNVFk2MC1sNUVKWkxibEF0WjU4aS10V0tlczlRX2ROUFJFcTNnSnZJTQ?oc=5)

2️⃣ **"중국, 6년간 AI에 2천조원 투자"…미중 간 '쩐의 전쟁' 가열**  
AI 기술 개발을 위한 중국의 대규모 투자가 미중 간 경쟁을 더욱 격화시키고 있습니다. [자세히 보기](https://news.google.com/rss/articles/CBMiT0FVX3lxTE5SOFY4M09KOG40VmxLX2xndTNoeFo0b3M0eENwd2JEbk1pSkpYdU1lQnQ3OUl0ZThTckdqNDRfblkzTklxTllDdjJzWE5IX1k?oc=5)

3️⃣ **[VC’s Pick] 투자 유치 성공한 ‘AI 의료 솔루션’들…글로벌 시장 정조준**  
AI 기반 의료 솔루션들이 글로벌 시장에서 주목받고 있습니다! [자세히 보기](https://news.google.com/rss/articles/CBMigAFBVV95cUxPdFktakY3cEZNVUUyYVY4bEY3QnZxdnpVeGxjLWYwYmhLNm9fYV90N2ZLVTBNODdHU2JQeUJFMzVWNFdsc1VfLXlqbWdjOC1PTG1nTHZYeHRoYmFrUFBPYWRGLWgxcWkxMGNRN1lnT0

In [ ]:
# claude
execute_agent(claude, tools, query, "claude")

[claude] 결과입니다.


검색 결과를 바탕으로 Instagram 게시글 형식으로 작성해 드리겠습니다.

---

📱 #AI투자 #테크트렌드 #글로벌경제

🔍 AI 투자 최신 동향: 글로벌 시장에서 주목받는 AI

1. 중국 빅테크 기업들의 AI 투자 러시 🇨🇳
   - '마윈 매직'으로 불리는 투자 열기
   - 알리바바, 텐센트 등 대형 IT 기업들 참여

2. 미중 AI 투자 경쟁 가열 🔥
   - 중국, 향후 6년간 AI에 약 2천조원 투자 계획
   - 미국과의 기술 패권 경쟁 심화

3. AI 의료 솔루션, 글로벌 시장 진출 🏥
   - 국내 AI 의료 기업들, 성공적인 투자 유치
   - 글로벌 시장 공략 본격화

4. AI 투자 전략의 새로운 관점 💡
   - 기존 AI 기술 활용한 차별화된 가치 창출 중요성 대두
   - 혁신적 비즈니스 모델 개발에 주목

5. 한미일 AI 협력 강조 🤝
   - 최태원 SK그룹 회장, AI 분야 한미일 협력 필요성 언급
   - 미국 투자 검토 중, 인센티브 중요성 강조

💬 AI 기술은 글로벌 경제의 핵심 동력으로 자리잡고 있습니다. 국가간, 기업간 경쟁이 치열해지는 가운데, 혁신적인 접근과 국제 협력의 중요성이 더욱 부각되고 있어요. 여러분은 AI 기술의 발전이 우리 삶에 어떤 영향을 미칠 것이라고 생각하시나요? 댓글로 여러분의 생각을 공유해주세요! 👇

#AI혁명 #투자트렌드 #기술혁신 #글로벌협력 #미래기술

---

이 Instagram 게시글은 AI 투자와 관련된 최신 뉴스를 요약하고, 주요 트렌드를 강조하며, 독자들의 참여를 유도하는 형식으로 작성되었습니다. 해시태그를 사용하여 검색 가능성을 높이고, 이모지를 활용하여 시각적 흥미를 더했습니다. 마지막으로 독자들에게 질문을 던져 댓글 참여를 유도하는 전략을 사용했습니다.


In [ ]:
# gemini
execute_agent(gemini, tools, query, "gemini")

[gemini] 결과입니다.
# AI 투자, 미래를 향한 현명한 선택? 🤔

AI 기술이 산업 전반을 뒤흔들고 있는 지금, AI 투자는 미래를 향한 현명한 선택이 될 수 있을까요? 최근 뉴스들을 통해 AI 투자의 현황과 전망을 살펴보세요.

* **중국, 6년간 AI에 2천조원 투자**: 미중 간 기술 패권 경쟁이 심화되는 가운데, 중국은 막대한 자금을 AI 분야에 쏟아붓고 있습니다. [관련 뉴스 링크](https://news.google.com/rss/articles/CBMiT0FVX3lxTE5SOFY4M09KOG40VmxLX2xndTNoeFo0b3M0eENwd2JEbk1pSkpYdU1lQnQ3OUl0ZThTckdqNDRfblkzTklxTllDdjJzWE5IX1k?oc=5)
* **투자 유치 성공한 'AI 의료 솔루션'**:  AI 기술은 의료 분야에도 혁신을 가져오고 있으며, 관련 스타트업들이 투자 유치에 성공하며 성장하고 있습니다. [관련 뉴스 링크](https://news.google.com/rss/articles/CBMigAFBVV95cUxPdFktakY3cEZNVUUyYVY4bEY3QnZxdnpVeGxjLWYwYmhLNm9fYV90N2ZLVTBNODdHU2JQeUJFMzVWNFdsc1VfLXlqbWdjOC1PTG1nTHZYeHRoYmFrUFBPYWRGLWgxcWkxMGNRN1lnT0dkS1RsR2VNdVNTNS1KWExIQw?oc=5)
* **최태원 "AI·에너지 분야 한미일 협력 필수"**: SK그룹 최태원 회장은 AI와 에너지 분야에서의 한미일 협력의 중요성을 강조했습니다. [관련 뉴스 링크](https://news.google.com/rss/articles/CBMidkFVX3lxTFBuQWtWWjlQcmxuS1BycGJoNzc2U0t6Ym5fR2RNWnhzN3RzR3A4WVZqUWpnYzVPRW1HVzZ0Q2FfX25Fc0ZneGItQzRBTmJveXlPUTFqMDdTemRvVF9LN1hwVmFEa2diSkNSSmxMZTQ0VzJQMTlxNGf

In [ ]:
# llama3.1 70B (Together.ai)
execute_agent(
    llama,
    tools,
    "Search AI related news and write it in Instagram post format. answer in Korean",#한글로 잘 안나옴
    "llama3.1 70B",
)

[llama3.1 70B] 결과입니다.
"AI News Alert!

Check out the latest news on AI from around the world!

National Assembly to Focus on ‘Save AI’ Mode - IT조선
AI가 제조업의 미래를 바꾼다 - SAP News
Korea's AI success hinges on innovation beyond technology - 네이트 뉴스
Coreline Soft Leads AI Medical Imaging Innovation at STR 2025 - KIPOST
Openai vs. Deepseek vs. Qwen : AI를 누가 지배 할 것인가? - 시사복지

Stay up-to-date with the latest developments in the world of AI! #AI #ArtificialIntelligence #Innovation"


In [ ]:
# llama3.1 8B (ollama)
execute_agent(ollama, tools, query, "llama3.1(Ollama)")

AttributeError: 'FieldInfo' object has no attribute 'chat'

In [ ]:
# qwen2.5 7B (ollama)
query = "AI 투자와 관련된 뉴스를 검색하고, 결과를 Instagram 게시글 형식으로 작성해 주세요. 한글로 답변하세요!"

execute_agent(qwen, tools, query, "qwen2.5(Ollama)")